In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_soom
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler

In [2]:
device = 'cpu'

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device='cpu'):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)
        
        return x


In [4]:
X, y = load_diabetes(return_X_y = True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
# print(X.shape)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))

torch_data = TensorDataset(torch.Tensor(X).to(device), torch.Tensor(y).to(device))
data_loader = DataLoader(torch_data, batch_size=1000)

In [8]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = pytorch_soom.Newton(model.parameters(), lr=1, model=model, hessian_approx=False, c1=1e-4, tau=0.5, line_search_method='backtrack')

all_loss = {}
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss[epoch+1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch+1] += loss
    all_loss[epoch+1] /= len(data_loader)
    print(', loss: {}'.format(all_loss[epoch+1].detach().numpy().item()))

epoch:  0, loss: 0.22615033388137817
epoch:  1, loss: 0.059771452099084854
epoch:  2, loss: 0.05810341611504555
epoch:  3, loss: 0.050128042697906494
epoch:  4, loss: 0.04632483050227165
epoch:  5, loss: 0.04360844939947128
epoch:  6, loss: 0.035846322774887085
epoch:  7, loss: 0.03239009529352188
epoch:  8, loss: 0.03190647438168526
epoch:  9, loss: 0.027748161926865578
epoch:  10, loss: 0.027609001845121384
epoch:  11, loss: 0.025527413934469223
epoch:  12, loss: 0.02480781264603138
epoch:  13, loss: 0.024347418919205666
epoch:  14, loss: 0.02412429265677929
epoch:  15, loss: 0.023808855563402176
epoch:  16, loss: 0.023439757525920868
epoch:  17, loss: 0.02314920537173748
epoch:  18, loss: 0.023126302286982536
epoch:  19, loss: 0.022728100419044495
epoch:  20, loss: 0.02230403944849968
epoch:  21, loss: 0.021877767518162727
epoch:  22, loss: 0.021844860166311264
epoch:  23, loss: 0.021375112235546112
epoch:  24, loss: 0.02125069685280323
epoch:  25, loss: 0.02109985053539276
epoch:  

In [9]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = pytorch_soom.Newton(model.parameters(), lr=1, model=model, hessian_approx=True, c1=1e-4, tau=0.5, line_search_method='backtrack')

all_loss = {}
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss[epoch+1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch+1] += loss
    all_loss[epoch+1] /= len(data_loader)
    print(', loss: {}'.format(all_loss[epoch+1].detach().numpy().item()))

epoch:  0, loss: 0.13586580753326416
epoch:  1, loss: 0.05672239884734154
epoch:  2, loss: 0.05422181636095047
epoch:  3, loss: 0.0540899895131588
epoch:  4, loss: 0.0533779039978981
epoch:  5, loss: 0.05321604385972023
epoch:  6, loss: 0.05292700603604317
epoch:  7, loss: 0.0523856058716774
epoch:  8, loss: 0.051473237574100494
epoch:  9, loss: 0.051403313875198364
epoch:  10, loss: 0.050941068679094315
epoch:  11, loss: 0.05079410597681999
epoch:  12, loss: 0.050296615809202194
epoch:  13, loss: 0.050226327031850815
epoch:  14, loss: 0.049812108278274536
epoch:  15, loss: 0.049738992005586624
epoch:  16, loss: 0.049449142068624496
epoch:  17, loss: 0.049125298857688904
epoch:  18, loss: 0.04873129725456238
epoch:  19, loss: 0.048401445150375366
epoch:  20, loss: 0.04737908020615578
epoch:  21, loss: 0.04734838753938675
epoch:  22, loss: 0.047150831669569016
epoch:  23, loss: 0.046923331916332245
epoch:  24, loss: 0.04684232175350189
epoch:  25, loss: 0.04569000005722046
epoch:  26, l